In [ ]:
import open3d as o3d
import pandas as pd
from sklearn import cluster

import read_pts

In [ ]:
demo = False

In [ ]:
pcd = o3d.io.read_point_cloud('point_cloud.pts')
if demo:
    o3d.visualization.draw_geometries([pcd])

In [ ]:
pcd_down = pcd.voxel_down_sample(voxel_size=0.001)
if demo:
    o3d.visualization.draw_geometries([pcd_down])

In [ ]:
df = read_pts.pcd_to_df(pcd_down)

In [ ]:
df_filter1 = df[df['luminance'] > 0.9]
read_pts.write_pts('output/filter1.pts', df_filter1)

In [ ]:
df_filter2 = df[(df['s'] < 0.05) & (df['v'] > 0.5)]
read_pts.write_pts('output/filter2.pts', df_filter2)

In [ ]:
df_filter3 = df[(df['s'] < 0.05) & (df['v'] > 0.5) & (df['z'] > 0.1)]
read_pts.write_pts('output/filter3.pts', df_filter3)

In [ ]:
pcd_filter3 = read_pts.df_to_pcd(df_filter3)

In [ ]:
model = cluster.KMeans(n_clusters=128, n_init='auto')
model.fit(df_filter3[['x', 'y', 'z']])

In [ ]:
clusters = o3d.geometry.PointCloud()
clusters.points = o3d.utility.Vector3dVector(model.cluster_centers_)
o3d.io.write_point_cloud('output/clusters.xyz', clusters, write_ascii=True)

In [ ]:
o3d.visualization.draw_geometries([pcd_filter3, clusters])